In [1]:
"""
https://www.tensorflow.org/beta/tutorials/quickstart/beginner
"""

import tensorflow as tf
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
import cv2
import numpy as np

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [3]:
def expand_dim(array, t_size):
    expanded_array = np.zeros((len(array), t_size[0], t_size[1], 1))
    for i, img in enumerate(array):
        img_temp = cv2.resize(img, dsize=t_size, interpolation=cv2.INTER_AREA)
        img_temp = np.expand_dims(img_temp, axis=-1)
        
        ## Nomalization
        expanded_array[i] = img_temp - .5
    
    return expanded_array

class DataFeeder():
    
    def __init__(self, dataset, label, batch_size):
        self.dataset = dataset
        self.label = label
        
        assert dataset.shape[0] == label.shape[0], ...
        "Dataset legnth: {}, Label length: {}".format(dataset.shape[0], label.shape[0])
        
        self.dataset_domain = dataset + tf.random.normal(dataset.shape, mean=0.0, stddev=.1)
        self.label_domain = label
        
        self.dataset_size = self.dataset.shape[0]
        self.batch_size = batch_size
        self.num_batch = int(np.floor(self.dataset_size / self.batch_size))
        self.idx_batch = 0
        
    def feed(self):
        
        first_idx = self.idx_batch*self.batch_size
        if self.idx_batch == self.num_batch:
            last_idx = self.dataset_size
        else:
            last_idx = first_idx + self.batch_size
        cur_batch_size = last_idx - first_idx 
            
        data = self.dataset[first_idx:last_idx]
        data_expaneded = expand_dim(data, (224, 224))
        data_expaneded = tf.cast(data_expaneded, tf.float32)
        
        domain_data = self.dataset_domain[first_idx:last_idx]
        domain_data_expaneded = expand_dim(domain_data, (224, 224))
        domain_data_expaneded = tf.cast(domain_data_expaneded, tf.float32)
        
        class_label = tf.one_hot(self.label[first_idx:last_idx], depth=10)
        domain_label = np.concatenate([np.tile(np.asarray([1, 0]), [cur_batch_size, 1]), np.tile(np.asarray([0, 1]), [cur_batch_size, 1])])
        
        self.idx_batch += 1
        if (self.idx_batch >= self.num_batch):
            self.shuffle()
            self.idx_batch = 0
        
        outputs = [data_expaneded, domain_data_expaneded, class_label, domain_label]
        
        return outputs
        
        
    def shuffle(self):
        labeled_data = list(zip(self.dataset, self.label))
        np.random.shuffle(labeled_data)
        (self.dataset, self.label) = zip(*labeled_data)
        
        labeled_domain_data = list(zip(self.dataset_domain, self.label_domain))
        np.random.shuffle(labeled_domain_data)
        (self.dataset_domain, self.label_domain) = zip(*labeled_data)
        
        self.idx_batch = 0


In [4]:
class EntryFlowConvLayer(tf.keras.Model):
    def __init__(self, input_shape):
        super(EntryFlowConvLayer, self).__init__()
        self.model = tf.keras.Sequential([
                        tf.keras.layers.Conv2D(filters=32, 
                            kernel_size=(3, 3), strides=(2, 2), use_bias=False,
                            padding='same', input_shape=input_shape),
                        tf.keras.layers.BatchNormalization(),
                        tf.keras.layers.Activation('relu'),
                        tf.keras.layers.Conv2D(filters=64, 
                            kernel_size=(3, 3), strides=(2, 2), use_bias=False,
                            padding='same'),
                        tf.keras.layers.BatchNormalization(),
                        tf.keras.layers.Activation('relu')])
    
    def call(self, inputs):
        outputs = self.model(inputs)
        return outputs

class EntryFlowResidualBlock(tf.keras.Model):
    def __init__(self, f_num):
        super(EntryFlowResidualBlock, self).__init__()
        
        self.block = tf.keras.models.Sequential([
                        tf.keras.layers.SeparableConv2D(filters=f_num, 
                            kernel_size=3, padding='same'),
                        tf.keras.layers.BatchNormalization(),
                        tf.keras.layers.Activation('relu'),        
                        tf.keras.layers.SeparableConv2D(filters=f_num, 
                            kernel_size=3, padding='same'),
                        tf.keras.layers.BatchNormalization(),
                        tf.keras.layers.MaxPooling2D(pool_size=3, 
                            strides=2, padding='same')])
        
        self.conv_layer = tf.keras.models.Sequential([
                            tf.keras.layers.Conv2D(filters=f_num, 
                                kernel_size=1,strides=2,padding='same'),
                            tf.keras.layers.BatchNormalization()])
                           
        
    def call(self, inputs):
        outputs1 = self.conv_layer(inputs)
        outputs2 = self.block(inputs)
        outputs = tf.add(outputs1, outputs2)
        return outputs
    
class MiddleFlowConvBlock(tf.keras.Model):
    def __init__(self, f_num):
        super(MiddleFlowConvBlock, self).__init__()
        
        self.block = tf.keras.models.Sequential([
                        tf.keras.layers.Activation('relu'), 
                        tf.keras.layers.SeparableConv2D(filters=f_num, 
                            kernel_size=3, padding='same'),
                        tf.keras.layers.BatchNormalization()])
        
    def call(self, inputs):
        outputs = self.block(inputs)
        return outputs
    
class MiddleFlowResidualBlock(tf.keras.Model):
    def __init__(self, f_num):
        super(MiddleFlowResidualBlock, self).__init__()
        
        self.block = tf.keras.models.Sequential([MiddleFlowConvBlock(f_num) 
                                                  for i in range(3)])

    def call(self, inputs):
        results = self.block(inputs)
        outputs = tf.add(results, inputs)

        return outputs
    
class ExitFlowResidualBlock(tf.keras.Model):
    def __init__(self):
        super(ExitFlowResidualBlock, self).__init__()
        
        self.block = tf.keras.models.Sequential([
                        tf.keras.layers.Activation('relu'), 
                        tf.keras.layers.SeparableConv2D(filters=728, 
                            kernel_size=3, padding='same'),
                        tf.keras.layers.BatchNormalization(),
                        tf.keras.layers.Activation('relu'), 
                        tf.keras.layers.SeparableConv2D(filters=1024, 
                            kernel_size=3, padding='same'),
                        tf.keras.layers.BatchNormalization(),
                        tf.keras.layers.MaxPooling2D(pool_size=3, 
                            strides=2, padding='same')])
        
        self.conv_layer = tf.keras.models.Sequential([
                            tf.keras.layers.Conv2D(filters=1024, 
                                kernel_size=1,strides=2,padding='same'),
                            tf.keras.layers.BatchNormalization()])
        
    def call(self, inputs):
        outputs1 = self.block(inputs)
        outputs2 = self.conv_layer(inputs)
        outputs = tf.add(outputs1, outputs2)
        return outputs
    
class ExitFlowConvLayer(tf.keras.Model):
    
    def __init__(self):
        super(ExitFlowConvLayer, self).__init__()
        self.model = tf.keras.models.Sequential([
                    tf.keras.layers.SeparableConv2D(filters=1536, 
                        kernel_size=3, padding='same'),
                    tf.keras.layers.BatchNormalization(),
                    tf.keras.layers.Activation('relu'), 
                    tf.keras.layers.SeparableConv2D(filters=2048, 
                        kernel_size=3, padding='same'),
                    tf.keras.layers.BatchNormalization(),
                    tf.keras.layers.Activation('relu'), 
                    tf.keras.layers.GlobalAveragePooling2D()])
        
    def call(self, inputs):
        outputs = self.model(inputs)
        return outputs
    
class FullyConnectedLayer(tf.keras.Model):
    def __init__(self, num_label, k_size):
        super(FullyConnectedLayer, self).__init__()
        self.model = tf.keras.models.Sequential([
                        tf.keras.layers.Dense(k_size, activation='relu'),
                        tf.keras.layers.Dense(k_size, activation='relu'),
                        tf.keras.layers.Dense(num_label, activation='softmax')])
        
    def call(self, inputs):
        outputs = self.model(inputs)
        return outputs
    
    
@tf.custom_gradient
def gradient_reversal_operator(x):
    def grad(dy):
        return -1 * dy
    return x, grad


In [5]:
class EntryFlow(tf.keras.Model):
    def __init__(self, input_shape):
        super(EntryFlow, self).__init__()
        self.model = tf.keras.models.Sequential([EntryFlowConvLayer(input_shape=input_shape),
                                               EntryFlowResidualBlock(128),
                                               EntryFlowResidualBlock(256),
                                               EntryFlowResidualBlock(728)])
        
    def call(self, inputs):
        outputs = self.model(inputs)
        return outputs
    
class MiddleFlow(tf.keras.Model):
    def __init__(self):
        super(MiddleFlow, self).__init__()
        self.model = tf.keras.models.Sequential([MiddleFlowResidualBlock(728)
                                                for i in range(8)])
        
    def call(self, inputs):
        outputs = self.model(inputs)
        return outputs  
    
class ExitFlow(tf.keras.Model):
    def __init__(self, num_label, k_size):
        super(ExitFlow, self).__init__()
        self.model = tf.keras.models.Sequential([ExitFlowConvLayer(), 
                                                FullyConnectedLayer(num_label, k_size)])
        
    def call(self, inputs):
        outputs = self.model(inputs)
        return outputs
    

# class Xception(tf.keras.Model):
#     def __init__(self, input_shape):
#         super(Xception, self).__init__()
# #         self.L1 = tf.keras.losses.CategoricalCrossentropy()
# #         self.optimizer = tf.keras.optimizers.Adam()
#         self.model = tf.keras.models.Sequential([EntryFlow(input_shape),
#                                            MiddleFlow(),
#                                            ExitFlow(10)])
    
#     def call(self, inputs):
#         outputs = self.model(inputs)
#         return outputs

class Xception():
    def __init__(self, input_shape):
#         self.L1 = tf.keras.losses.CategoricalCrossentropy()
#         self.optimizer = tf.keras.optimizers.Adam()
        self.feature_extractor = tf.keras.models.Sequential([EntryFlow(input_shape), MiddleFlow(), ExitFlowConvLayer()])
        self.label_predictor = FullyConnectedLayer(10, 1024)
        self.domain_classifier = FullyConnectedLayer(2, 128)
        
        self.path1 = tf.keras.models.Sequential([self.feature_extractor, self.label_predictor])
        self.path2 = tf.keras.models.Sequential([self.feature_extractor, self.domain_classifier])
        
        self.optimizer = tf.keras.optimizers.Adagrad()
        self.loss_function = tf.keras.losses.CategoricalCrossentropy()
        self.class_metric = tf.keras.metrics.CategoricalAccuracy()
        self.domain_metric = tf.keras.metrics.CategoricalAccuracy()
        self.train_loss = tf.keras.metrics.Mean()
    
    def fit(self, class_inputs, domain_inputs, true):
        with tf.GradientTape(persistent=True) as tape:
            prediction = self.predict(class_inputs, domain_inputs)
            loss = self.calculate_loss(true, prediction)
        self.update_gradient(tape, loss)
        del tape
        return self.calculate_log(loss, true[0], true[1], prediction[0], prediction[1])
    
    @tf.function
    def predict(self, class_inputs, domain_inputs):
        predicted_label = self.path1(class_inputs)   
        inputs = tf.concat([class_inputs, domain_inputs], axis=0)
        predicted_domain = self.path2(inputs)        
        prediction = (predicted_label, predicted_domain)
        return prediction
    
    
    def calculate_loss(self, true, prediction):
        L1 = self.loss_function(true[0], prediction[0])
        L2 = self.loss_function(true[1], prediction[1])
        loss = (L1, L2)
        return loss

    def update_gradient(self, tape, loss):
        grad_fy = tape.gradient(loss[0], self.feature_extractor.trainable_variables)
        grad_y = tape.gradient(loss[0], self.label_predictor.trainable_variables)
        grad_dy = tape.gradient(loss[1], self.feature_extractor.trainable_variables)
        grad_d = tape.gradient(loss[1], self.domain_classifier.trainable_variables)
        
#         print(grad_fy)
#         print(grad_y)
#         print(grad_dy)
#         print(grad_d)
        
        self.optimizer.apply_gradients(zip(grad_y, self.label_predictor.trainable_variables))
        self.optimizer.apply_gradients(zip(grad_d, self.domain_classifier.trainable_variables))
        self.optimizer.apply_gradients(zip(grad_fy, self.feature_extractor.trainable_variables))
        self.optimizer.apply_gradients(zip(grad_dy, self.feature_extractor.trainable_variables))

        
    def calculate_log(self, loss, c_label, d_label, c_pred, d_pred):
#         self.train_loss(loss[0] + loss[1])
        self.train_loss(loss[0] + loss[1])
        self.class_metric(c_label, c_pred)
        self.domain_metric(d_label, d_pred)
        log = (self.train_loss.result(), self.class_metric.result(), self.domain_metric.result())
        return log

In [6]:
data_feeder = DataFeeder(x_train, y_train, 40)

In [7]:
data_feeder.shuffle()
# plt.imshow(data_feeder.dataset[0])
# data_feeder.label[0]
# a, b, c, d = data_feeder.feed()

In [8]:
model = Xception(input_shape=(224, 224, 1))

In [9]:
[data, domain_data, class_label, domain_label] = data_feeder.feed()
(class_pred, domain_pred) = model.predict(data, domain_data)

In [ ]:
EPOCHS = 1

for epoch in range(EPOCHS):
    for i in tqdm(range(data_feeder.num_batch)):        
        [data, domain_data, class_label, domain_label] = data_feeder.feed()
#         (class_pred, domain_pred) = model.predict(data, domain_data)
#         loss = model.calculate_loss((class_label, domain_label), (class_pred, domain_pred))        
#         (m_loss, c_acc, d_acc) = model.calculate_log(loss, class_label, domain_label, class_pred, domain_pred)
        
        (m_loss, c_acc, d_acc) = model.fit(data, domain_data, (class_label, domain_label))

    #     for test_images, test_labels in test_ds:
    #         test_step(test_images, test_labels)
        if i % 50 == 0:
            template = 'Loss: {}, Class Accuracy: {}, Domain Accuracy: {}'
            print(template.format(m_loss, c_acc*100, d_acc*100))


W0625 20:18:05.971215 22284 deprecation.py:323] From c:\users\jw\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\math_grad.py:1220: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [ ]:
@tf.function
def train_step(X, Y):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

In [ ]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test, y_test)